
## Train Inverse Model (Synthetic Data, from output constraints optimization)

* Synthetic Data gen: gradient based input optimization from constrained outputs 
* invert inputs and outputs and train

## Web application 

Run interactively on the web at: 


## Inputs


* desired output requirements 

Raceway flame temp > 1900K.....(19, 'o_fta')
 
Hot metal temp > 1700K........(21, 'o_hmt')

Top gas temp > 100C,,,,,,,,,,,(23, 'o_tgt')

production rate ..............(39, 'o_prod_rt')


## Outputs



(2, 'i_h2i_rate')

(3, 'i_h2_temp')

(4, 'i_ngi_rate')

(6, 'i_pci_rate')

(11, 'i_o2_volfract')

(12, 'i_hbtemp')

(13, 'i_wind_rt')











## Installs


In [1]:

## !pip install xgboost
## !pip install onnxmltools
## !pip install onnxruntime
## !pip install pandas
## !pip install scikit-learn
## !pip install matplotlib
## !pip install seaborn
## !pip install mlxtend
## ! pip install skl2onnx



## Imports


In [2]:

import torch
import numpy as np
import pandas as pd
import sklearn
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import math
import seaborn as sns
import random
import functorch


import torch.optim as optim


from numpy.random import normal
from scipy.stats import norm
import scipy.stats as stats
from numpy import hstack
from numpy import vstack
from numpy import exp
from sklearn.neighbors import KernelDensity

##from typing import Optional

from torch import Tensor


from torch.utils.data import TensorDataset, DataLoader

from mlxtend.plotting import heatmap

## coefficient of determination (R**2)
from sklearn.metrics import r2_score

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [3]:

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster


from zipfile import ZipFile
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)


# Import the ONNX runtime environment
import onnxruntime as rt



In [4]:

import PDFshapingUtils as PDF_tk



## Instantiate PDF shaping and regression class


In [5]:

PDFshapingOBJ = PDF_tk.PDFshapingUtils()



## Initialize parameters


In [6]:

PDFshapingOBJ.N_error_range = 5         ## 20
PDFshapingOBJ.mean_impulse  = 0.0       ## -7.0      ## 0.0
PDFshapingOBJ.std_impulse   = 0.001     ## 1.0
PDFshapingOBJ.kde_std       = 0.5       ## 0.1       ## 0.5      ## 3.0  ## smaller is better approximation?
PDFshapingOBJ.bandwidth     = 0.2       ## 0.2->more jagged, 2.0->more smooth


In [7]:

PDFshapingOBJ.initializeImpulseGaussian()



## Load Data


In [8]:

df_synthetic_data_for_inverse = PDFshapingOBJ.load_synth_folder_to_pandas("SyntheticCorpus", flatten=True)


In [9]:

PDFshapingOBJ.CFD_raw_data = df_synthetic_data_for_inverse 


In [10]:

PDFshapingOBJ.headers_list = PDFshapingOBJ.CFD_raw_data.columns.values.tolist()


In [11]:

PDFshapingOBJ.CFD_raw_data


,clamp_min_0,clamp_min_1,clamp_min_2,clamp_min_3,clamp_min_4,clamp_min_5,clamp_min_6,final_input_0,final_input_1,final_input_2,...,lower_bounds_glob_2,lower_bounds_glob_3,final_output_0,final_output_1,final_output_2,final_output_3,upper_bounds_glob_0,upper_bounds_glob_1,upper_bounds_glob_2,upper_bounds_glob_3
0,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,30.913000,605.179993,70.586998,...,12.201427,8336.685547,2412.514893,807.518005,34.897999,9327.898438,2457.082764,945.653198,43.632729,9949.238281
1,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,8.196000,849.021973,84.723000,...,136.951645,6527.677734,2218.454102,1723.489990,117.444000,7630.222168,2384.780273,1846.199707,161.503098,8197.164062
2,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,19.176001,385.032013,47.999001,...,70.967445,8830.876953,2063.924072,555.703979,71.166000,9310.989258,2061.797363,814.313354,80.050781,9196.826172
3,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,10.705000,609.044983,39.851002,...,72.513382,9474.287109,2383.951904,1627.159058,77.302002,10752.974609,2449.659424,2199.852051,110.303909,10992.480469
4,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,19.545000,775.119995,68.838997,...,106.992905,7615.858398,2229.862061,1464.323975,102.764999,8250.154297,2437.204590,1541.996338,135.790146,8403.231445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,23.858000,759.765015,57.305000,...,93.745247,8092.908691,2377.839111,1478.128052,92.150002,8597.575195,2428.693604,2075.100830,137.209122,9675.949219
156,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,15.391000,946.646973,77.273003,...,66.551659,8981.023438,2572.766113,1936.387939,73.183998,9273.188477,2670.626953,2184.010742,96.083076,9712.229492
157,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,21.216999,517.351990,47.403999,...,88.621223,9398.181641,2091.215088,504.355988,91.134003,10011.727539,2163.517334,924.842590,106.366951,10341.786133
158,0.0,0.0,0.0,0.0,0.0,1150.0,100.0,20.695000,735.875000,60.167000,...,78.929237,7034.716797,2237.495117,1388.083984,97.424004,7873.049805,2335.684570,1612.003174,118.919418,7760.502441


In [12]:

PDFshapingOBJ.print_headers_list()


['clamp_min_0', 'clamp_min_1', 'clamp_min_2', 'clamp_min_3', 'clamp_min_4', 'clamp_min_5', 'clamp_min_6', 'final_input_0', 'final_input_1', 'final_input_2', 'final_input_3', 'final_input_4', 'final_input_5', 'final_input_6', 'clamp_max_0', 'clamp_max_1', 'clamp_max_2', 'clamp_max_3', 'clamp_max_4', 'clamp_max_5', 'clamp_max_6', 'lower_bounds_glob_0', 'lower_bounds_glob_1', 'lower_bounds_glob_2', 'lower_bounds_glob_3', 'final_output_0', 'final_output_1', 'final_output_2', 'final_output_3', 'upper_bounds_glob_0', 'upper_bounds_glob_1', 'upper_bounds_glob_2', 'upper_bounds_glob_3']
33
(0, 'clamp_min_0')
(1, 'clamp_min_1')
(2, 'clamp_min_2')
(3, 'clamp_min_3')
(4, 'clamp_min_4')
(5, 'clamp_min_5')
(6, 'clamp_min_6')
(7, 'final_input_0')
(8, 'final_input_1')
(9, 'final_input_2')
(10, 'final_input_3')
(11, 'final_input_4')
(12, 'final_input_5')
(13, 'final_input_6')
(14, 'clamp_max_0')
(15, 'clamp_max_1')
(16, 'clamp_max_2')
(17, 'clamp_max_3')
(18, 'clamp_max_4')
(19, 'clamp_max_5')
(20, 'c

In [13]:
'''


PDFshapingOBJ.list_of_selected_column_names = ['i_h2i_rate','i_h2_temp','i_ngi_rate','i_ng_temp','i_pci_rate',
                    'i_wpi_rate','i_o2_volfract',
                    'i_hbtemp','i_wind_rt','o_prod_rt', 'o_tgt', 'o_hmt','o_fta']



PDFshapingOBJ.print_correlation_coefficients()

'''


"\n\n\nPDFshapingOBJ.list_of_selected_column_names = ['i_h2i_rate','i_h2_temp','i_ngi_rate','i_ng_temp','i_pci_rate',\n                    'i_wpi_rate','i_o2_volfract',\n                    'i_hbtemp','i_wind_rt','o_prod_rt', 'o_tgt', 'o_hmt','o_fta']\n\n\n\nPDFshapingOBJ.print_correlation_coefficients()\n\n"

In [14]:

PDFshapingOBJ.convert_pd_data_to_numpy()


[[    0.             0.             0.         ...   945.65319824
     43.63272858  9949.23828125]
 [    0.             0.             0.         ...  1846.19970703
    161.50309753  8197.1640625 ]
 [    0.             0.             0.         ...   814.31335449
     80.05078125  9196.82617188]
 ...
 [    0.             0.             0.         ...   924.84259033
    106.36695099 10341.78613281]
 [    0.             0.             0.         ...  1612.00317383
    118.91941833  7760.50244141]
 [    0.             0.             0.         ...  1476.78222656
    117.44421387  9028.05371094]]
(160, 33)



## Select inputs and outputs


In [15]:



'''
(21, 'lower_bounds_glob_0')
(22, 'lower_bounds_glob_1')
(23, 'lower_bounds_glob_2')
(24, 'lower_bounds_glob_3')

(29, 'upper_bounds_glob_0')
(30, 'upper_bounds_glob_1')
(31, 'upper_bounds_glob_2')
(32, 'upper_bounds_glob_3')
'''




PDFshapingOBJ.gen_X_y_for_selected_indeces(  
                   inputs = [ 21,22,23,24,    29,30,31,32 ] ,  
                   outputs= [  7, 8, 9, 10, 11, 12, 13 ]   
)


8
7


In [16]:

print(PDFshapingOBJ.X.shape)
print(PDFshapingOBJ.y.shape)


(160, 8)
(160, 7)


In [17]:

PDFshapingOBJ.random_seed = int( random.random() * 100  )         ## defautl is 42


In [18]:

PDFshapingOBJ.split_np_data_train_test(selected_test_size=0.2)


(128, 8)
(32, 8)
(128, 7)
(32, 7)


In [19]:

PDFshapingOBJ.convert_dataset_from_np_to_torch()


In [20]:

PDFshapingOBJ.standardize_X_scales()
PDFshapingOBJ.standardize_y_scales()


In [21]:

PDFshapingOBJ.gen_Dataloader_train()

PDFshapingOBJ.train_dl


In [22]:

n_inputs  = 8
n_outputs = 7

class ResidualNet(nn.Module):
    def __init__(self, x_means, x_deviations, y_means, y_deviations, dropout_rate=0.1, device='cuda'):
        
        super().__init__()
        self.device = device
        
        self.x_means      = x_means.to(self.device)
        self.x_deviations = x_deviations.to(self.device)
        self.y_means      = y_means.to(self.device)
        self.y_deviations = y_deviations.to(self.device)

     
        self.fc1  = nn.Linear(n_inputs, 64)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(64, 64)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(64, n_outputs)

        # If input_dim ≠ 64, project for residual addition
        self.input_proj = nn.Linear(n_inputs, 64) if n_inputs != 64 else nn.Identity()
        
        # Move model to device
        self.to(self.device)
        

    def forward(self, x):
        x  = x.to(self.device)
        x  = (x - self.x_means) / self.x_deviations
        x0 = self.input_proj(x)
        x1 = self.dropout1(self.act1(self.fc1(x)))
        x2 = self.dropout2(self.act2(self.fc2(x1)))
        x_res = x2 + x0
        y_scaled = self.fc3(x_res)
        y_descaled = y_scaled * self.y_deviations + self.y_means
        
        return y_descaled, y_scaled
    
    


In [23]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model     = ResidualNet(
             PDFshapingOBJ.x_means, 
             PDFshapingOBJ.x_deviations, 
             PDFshapingOBJ.y_means, 
             PDFshapingOBJ.y_deviations,
             device=device  # pass device into the model
    
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.MSELoss()




In [24]:

model.train()

for epoch in range(1000):
    for xb, yb in PDFshapingOBJ.train_dl:
        
        xb = xb.to(model.device)
        yb = yb.to(model.device)
        
        optimizer.zero_grad()
        pred_descaled, pred_scaled = model(xb)
        loss = loss_fn(pred_scaled, yb)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(epoch, ".....", loss.item() ) 
    


    

0 ..... 0.8415763974189758
100 ..... 0.22225098311901093
200 ..... 0.14222905039787292
300 ..... 0.09135176986455917
400 ..... 0.1499253213405609
500 ..... 0.06743141263723373
600 ..... 0.0501396618783474
700 ..... 0.05051017925143242
800 ..... 0.03698539361357689
900 ..... 0.03827572986483574


In [25]:

print("=== SANITY CHECK: SCALING ===")
print("y_means:", PDFshapingOBJ.y_means)
print("y_deviations:", PDFshapingOBJ.y_deviations)

print("===============================================")

print("x_means:", PDFshapingOBJ.x_means)
print("x_deviations:", PDFshapingOBJ.x_deviations)



=== SANITY CHECK: SCALING ===
y_means: tensor([[  18.9724,  739.1686,   56.9263,   62.2738,   28.4872, 1395.5881,
          194.1112]])
y_deviations: tensor([[  8.1668, 199.8139,  25.4188,  31.3017,   6.3484,  54.6121,  32.5499]])
x_means: tensor([[2249.3965, 1157.8499,   72.0484, 8123.1006, 2404.3506, 1555.1772,
          100.1305, 9152.0352]])
x_deviations: tensor([[ 176.2738,  586.9822,   42.0676, 1273.3890,  189.1688,  600.4334,
           43.0604, 1338.2035]])



## Test model


In [26]:

print( PDFshapingOBJ.X_test_tr[0] )
print( PDFshapingOBJ.y_test_tr[0] )


tensor([2707.0635, 2021.4675,   58.6415, 7458.4072, 2779.5774, 2460.7207,
          84.6981, 7890.9263])
tensor([  13.5500,  981.6370,   30.6540,   19.1640,   31.5310, 1452.5420,
         197.0270])


In [27]:


print( PDFshapingOBJ.X_test_tr.shape )
print( PDFshapingOBJ.y_test_tr.shape )

with torch.no_grad():
    
    pred_test_descaled, pred_test_scaled = model(  PDFshapingOBJ.X_test_tr  )


torch.Size([32, 8])
torch.Size([32, 7])


In [28]:


y_true = PDFshapingOBJ.y_test_tr.detach().cpu().numpy()

y_pred = pred_test_descaled.detach().cpu().numpy()  # or use descaled pair

mae  = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
rmse = mean_squared_error( y_true, y_pred, squared=False, multioutput='raw_values')
r2   = r2_score(           y_true, y_pred, multioutput='raw_values')

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)
print("Overall averages:", 
      mae.mean(), rmse.mean(), r2.mean())



MAE: [  5.175151  128.34592    17.226624   15.871365    2.0290265  23.346806
   7.773594 ]
RMSE: [  7.1440663 176.14488    22.407133   20.728752    2.5812876  32.1756
   9.214191 ]
R2: [0.26235676 0.44771771 0.25511057 0.54981496 0.83291025 0.68454721
 0.88031904]
Overall averages: 28.538355 38.627987 0.5589680709116823


In [29]:

r2   = r2_score(           y_true, y_pred, multioutput='raw_values')
r2


array([0.26235676, 0.44771771, 0.25511057, 0.54981496, 0.83291025,
       0.68454721, 0.88031904])

In [30]:

r2.mean()


0.5589680709116823

In [31]:

print('Testing R**2 - descaled:', r2_score( y_pred, y_true   )  )


Testing R**2 - descaled: 0.3801857671222973


In [32]:

print('Testing R**2 - descaled:', r2_score( y_true,  y_pred   )  )


Testing R**2 - descaled: 0.5589680709116823


In [33]:

PDFshapingOBJ.r2_score_torch(PDFshapingOBJ.y_test_tr.to(PDFshapingOBJ.device), pred_test_descaled).mean()


tensor(0.5590, device='cuda:0')


## ONNX


In [34]:

import torch

# Your trained model
model.eval().float()


dummy_input = torch.randn(1, 8, dtype=torch.float32)

# Export to ONNX (single output called "output1")
torch.onnx.export(
    model,
    dummy_input,
    "ONNXmodels/resNet_Inverse_syntheticData.onnx",
    input_names=["input1"],
    output_names=["output1"],
    opset_version=15,              # fine for onnxruntime-web
    do_constant_folding=True,
    dynamic_axes={
        "input1": {0: "batch"},
        "output1": {0: "batch"}
    }
)
print("ONNX model saved")



ONNX model saved



## XGBoost


In [35]:

regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)


In [36]:

regressor.fit(PDFshapingOBJ.X_train, PDFshapingOBJ.y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [37]:

print( len(PDFshapingOBJ.X_train  ))
print( len(PDFshapingOBJ.y_train  ))


128
128


In [38]:

print( PDFshapingOBJ.X_train.shape  )
print( PDFshapingOBJ.y_train.shape  )


(128, 8)
(128, 7)


In [39]:

y_pred = regressor.predict(PDFshapingOBJ.X_test )


In [40]:

y_pred


array([[   9.853494 , 1025.2375   ,   39.82202  ,   62.429737 ,
          29.891685 , 1403.638    ,  193.89433  ],
       [  11.858862 ,  735.5758   ,   35.643517 ,   13.546301 ,
          25.65001  , 1469.0774   ,  197.93553  ],
       [  17.351114 ,  837.4612   ,   48.267517 ,   34.116962 ,
          28.932585 , 1429.26     ,  191.36783  ],
       [  14.115894 ,  595.33746  ,   61.09657  ,   60.749134 ,
          21.056534 , 1396.7827   ,  232.1715   ],
       [  11.953241 ,  923.4205   ,   46.41536  ,   47.332912 ,
          31.407564 , 1396.5312   ,  172.27466  ],
       [  24.193398 ,  746.6952   ,   61.659023 ,   26.2101   ,
          26.98408  , 1445.4872   ,  194.8274   ],
       [  12.364178 ,  734.36926  ,   40.15349  ,   38.998226 ,
          29.203281 , 1435.1046   ,  188.46077  ],
       [  22.603748 ,  680.8327   ,   82.90083  ,   99.1888   ,
          39.324715 , 1338.7142   ,  144.59378  ],
       [  25.14637  ,  564.31213  ,   46.30333  ,   34.017956 ,
          24.937

In [41]:

r2_score( PDFshapingOBJ.y_test, y_pred )


0.5599516221146192